In [27]:
import os
import sys
import time
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.constants import c
from lib.loadlattice import prepareTwiss
from lib.IBSfunctions import NagaitsevIBS

from blond.beam.beam import Beam, Positron
from blond.input_parameters.ring import Ring
from blond.monitors.monitors import BunchMonitor
from blond.input_parameters.rf_parameters import RFStation
from blond.beam.profile import CutOptions, FitOptions, Profile
from blond.trackers.tracker import RingAndRFTracker, FullRingAndRF
from blond.beam.distributions import bigaussian, matched_from_distribution_function

### CLIC Parameters

In [28]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

# !~~~~~~~~~~~~ Bunch parameters ~~~~~~~~~~~! #
N_b = config['bunch_intensity'] # Number of particles; [1]
N_p = int(config['n_mparts'])   # Number of macro-particles; [1]
tau_0 = config['tau']           # 4 sigma bunch length, 4 sigma [s]

# !~~~~~~~ Machine and RF parameters ~~~~~~~! #
C = 427.5                       # Machine circumference [m]
p = config['energy'] * 1e9      # Synchronous momentum [eV/c]
h = config['h']                 # Harmonic number
phi = 0.                        # RF synchronous phase
Vrf = config['V0max'] * 1e9     # RF voltage [V]

gamma_t = 87.70580              # Transition gamma
alpha   = 1 / gamma_t**2        # First order mom. comp. factor

# !~~~~~~~~~~~~ Tracking details ~~~~~~~~~~~! #
N_t    = config['N_turns']      # Number of turns to track
N_ibs  = config['IBS_stp']      # Number of turns to update IBS
N_mtch = 1000                   # Number of turns for matching
dt_plt = 50

print(f'Circumference: {C} [m],\nHarmonic number: {h}, \nVrf: {Vrf} [V], \ngamma transition: {gamma_t}, \nmomentum compaction factor: {alpha}\n')
print(f'Bunch intensity: {N_b} electrons,\nNumber of macroparticles: {N_p}, \nBunch length: {tau_0} ns, \nEnergy: {p * 1e-9} [GeV]')

Circumference: 427.5 [m],
Harmonic number: 2852.0, 
Vrf: 4500000.0 [V], 
gamma transition: 87.7058, 
momentum compaction factor: 0.00013000000572348426

Bunch intensity: 4.4e9 electrons,
Number of macroparticles: 1000000, 
Bunch length: 1.72e-11 ns, 
Energy: 2.8600000000000003 [GeV]


### Simulation Setup

In [29]:
wrkDir = os.getcwd()
print(f"Working directory is {wrkDir}")
if not os.path.isdir(os.path.join(wrkDir,'output')):
    os.mkdir("output")
    print(f"Created output folder {os.path.join(wrkDir,'output')}")


# !~~~~~~~ Define General Parameters ~~~~~~~! #
ring = Ring(C, alpha, p, Positron(), N_t + N_mtch)
print("Ring initialized...")


# !~~~~~~ Define RF Station Parameters ~~~~~! #
rf = RFStation(ring, [h], [Vrf], [phi])
print("RF station initialized...")


# !~~~~~~ Define beam and distribution ~~~~~! #
beam = Beam(ring, N_p, N_b)

profile = Profile(beam, CutOptions(n_slices=100, cut_left=0, 
                    cut_right=rf.t_rf[0, 0]), 
                    FitOptions=FitOptions(fit_option='rms'))

# !~~~~~~~~~~~~~~~~ Trackers ~~~~~~~~~~~~~~~! #
rf_station_tracker = RingAndRFTracker(rf, beam, Profile=profile)
tracker = FullRingAndRF([rf_station_tracker])


# !~~~~~~~~~~~~~~~~ Matching ~~~~~~~~~~~~~~! #
# matched_from_distribution_function(beam, tracker,
#     distribution_type = 'gaussian', bunch_length = tau_0,
#     distribution_variable = 'Hamiltonian', bunch_length_fit = 'gaussian', 
#     n_iterations= 10)

# matched_from_distribution_function(beam, tracker,
#     distribution_type = 'binomial', bunch_length = tau_0, distribution_exponent = 2,
#     distribution_variable = 'Hamiltonian', bunch_length_fit = 'gaussian', 
#     n_iterations= 10)

bigaussian(ring, rf, beam, tau_0 /4.)#, 0.0014748188 * beam.energy)
        #    0.0011220955 * beam.energy)

profile.track()
print(f'4sigma bunch length requested = {tau_0} [s]')
print(f'4sigma bunch length from profile = {profile.bunchLength} [s]')
print(f'4sigma bunch length from np.std = {np.std(beam.dt) * 4} s')
print(f'Energy spread = {np.std(beam.dE)/beam.energy}')


# !~~~~~ Define what to save in file ~~~~~~! #
bunchmonitor = BunchMonitor(ring, rf, beam, wrkDir + '/output/lhc_output_data', 
                            Profile=profile)

# !~~~~~~~~~~~~ Accelerator map ~~~~~~~~~~~~! #
map_ = [tracker] + [profile] + [bunchmonitor]
print("Map set")
print("")

for turn in range(1, N_mtch+1):
    if turn % 100 == 0: print(f'Turn = {turn}')
   # Track
    tracker.track()
    profile.track()
print(f'4sigma bunch length requested = {tau_0} [s]')
print(f'4sigma bunch length from profile = {profile.bunchLength} [s]')
print(f'4sigma bunch length from np.std = {np.std(beam.dt) * 4} s')
print(f'Energy spread = {np.std(beam.dE)/beam.energy}')

Working directory is /Users/michalis/Work/Fellow/IBS_BLonD/CLIC_DRs
Ring initialized...
RF station initialized...
4sigma bunch length requested = 1.72e-11 [s]
4sigma bunch length from profile = 1.813297145674646e-11 [s]
4sigma bunch length from np.std = 1.7197195711825157e-11 s
Energy spread = 0.0014048258374507962
Map set

Turn = 100
Turn = 200
Turn = 300
Turn = 400
Turn = 500
Turn = 600
Turn = 700
Turn = 800
Turn = 900
Turn = 1000
4sigma bunch length requested = 1.72e-11 [s]
4sigma bunch length from profile = 1.791458331264441e-11 [s]
4sigma bunch length from np.std = 1.694929797869764e-11 s
Energy spread = 0.001426080180238634


### Run Analytic IBS

In [30]:
twiss = prepareTwiss(config['twissfile'])
twiss['slip'] = rf.eta_0[0]

IBS = NagaitsevIBS()
IBS.set_beam_parameters(beam)
IBS.set_optic_functions(twiss)

In [31]:
analytic = True
if analytic:
    emit_x = config['epsn_x'] / beam.gamma / beam.beta
    emit_y = config['epsn_y'] / beam.gamma / beam.beta
    bunch_length  = np.std(beam.dt) * c * beam.beta #profile.bunchLength * c * beam.beta / 4.
    sigma_epsilon = np.std(beam.dE[beam.id > 0]) / beam.energy
    sigma_delta   = sigma_epsilon / IBS.betar**2

    evolution = {'time': np.zeros(int(N_t / N_ibs)), 
                 'epsn_x': np.zeros(int(N_t / N_ibs)),
                 'epsn_y': np.zeros(int(N_t / N_ibs)),
                 'tau_ns': np.zeros(int(N_t / N_ibs)),
                 'deltaE': np.zeros(int(N_t / N_ibs))}
    
    indx = 0
    for turn in range(0, N_t):
        if turn % 1000 == 0: print(f'Turn = {turn}')

        if turn % N_ibs == 0:
            IBS.growth_rates(emit_x, emit_y, sigma_delta, bunch_length)
            evolution['time'][indx]  = turn / IBS.frev
            evolution['epsn_x'][indx] = emit_x * beam.beta * beam.gamma
            evolution['epsn_y'][indx] = emit_y * beam.beta * beam.gamma
            evolution['tau_ns'][indx] = bunch_length / c / beam.beta * 4
            evolution['deltaE'][indx] = sigma_epsilon
            indx += 1

        emit_x, emit_y, sigma_delta, bunch_length = IBS.emittance_evolution(emit_x, emit_y, sigma_delta, 
                                                                            bunch_length, 1 / ring.f_rev[0])
        sigma_epsilon = sigma_delta * beam.beta**2

    df = pd.DataFrame(evolution)
    df.to_parquet('output/IBS_output_python.parquet')


Turn = 0
Turn = 1000
Turn = 2000
Turn = 3000
Turn = 4000
Turn = 5000
Turn = 6000
Turn = 7000
Turn = 8000
Turn = 9000
Turn = 10000
Turn = 11000
Turn = 12000
Turn = 13000
Turn = 14000
Turn = 15000


/opt/homebrew/Caskroom/miniforge/base/envs/blond/lib/python3.9/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


### Run IBS Tracking

In [32]:
emit_x = config['epsn_x'] / beam.gamma / beam.beta
emit_y = config['epsn_y'] / beam.gamma / beam.beta

evolution = {'time': np.zeros(int(N_t / N_ibs)),
             'epsn_x': np.zeros(int(N_t / N_ibs)),
             'epsn_y': np.zeros(int(N_t / N_ibs)),
             'tau_ns': np.zeros(int(N_t / N_ibs)),
             'deltaE': np.zeros(int(N_t / N_ibs))}
indx = 0
for turn in range(1, N_t+1):
    if ((turn-1) % N_ibs == 0):
        print(f'Turn = {turn}')
        IBS.calculate_longitudinal_kick(emit_x, emit_y, beam)
        evolution['time'][indx]  = (turn-1)/ IBS.frev
        evolution['epsn_x'][indx] = emit_x * IBS.betar * IBS.gammar
        evolution['epsn_y'][indx] = emit_y * IBS.betar * IBS.gammar
        evolution['tau_ns'][indx] = np.std(beam.dt) * 4 #profile.bunchLength
        evolution['deltaE'][indx] = np.std(beam.dE[beam.id > 0])/beam.energy
        indx += 1


    IBS.track(profile, beam)
    emit_x, emit_y = IBS.emittance_evolution_2D(emit_x, emit_y, 1 / ring.f_rev[0])


   # Track
    tracker.track()
    profile.track()

    # beam.losses_separatrix(ring, rf)
    # beam.losses_longitudinal_cut(0., rf.t_rf[0, 0])

df = pd.DataFrame(evolution)
df.to_parquet("output/IBS_output_BLonD.parquet")

Turn = 1
Turn = 51
Turn = 101
Turn = 151
Turn = 201
Turn = 251
Turn = 301
Turn = 351
Turn = 401
Turn = 451
Turn = 501
Turn = 551
Turn = 601
Turn = 651
Turn = 701
Turn = 751
Turn = 801
Turn = 851
Turn = 901
Turn = 951
Turn = 1001
Turn = 1051
Turn = 1101
Turn = 1151
Turn = 1201
Turn = 1251
Turn = 1301
Turn = 1351
Turn = 1401
Turn = 1451
Turn = 1501
Turn = 1551
Turn = 1601
Turn = 1651
Turn = 1701
Turn = 1751
Turn = 1801
Turn = 1851
Turn = 1901
Turn = 1951
Turn = 2001
Turn = 2051
Turn = 2101
Turn = 2151
Turn = 2201
Turn = 2251
Turn = 2301
Turn = 2351
Turn = 2401
Turn = 2451
Turn = 2501
Turn = 2551
Turn = 2601
Turn = 2651
Turn = 2701
Turn = 2751
Turn = 2801
Turn = 2851
Turn = 2901
Turn = 2951
Turn = 3001
Turn = 3051
Turn = 3101
Turn = 3151
Turn = 3201
Turn = 3251
Turn = 3301
Turn = 3351
Turn = 3401
Turn = 3451
Turn = 3501
Turn = 3551
Turn = 3601
Turn = 3651
Turn = 3701
Turn = 3751
Turn = 3801
Turn = 3851
Turn = 3901
Turn = 3951
Turn = 4001
Turn = 4051
Turn = 4101
Turn = 4151
Turn = 4201
Tur

In [8]:
len(beam.dE[beam.id > 0])

1000000

In [9]:
# 
# !~~~~~~ Define beam and distribution ~~~~~! #
beam = Beam(ring, N_p, N_b)

profile = Profile(beam, CutOptions(n_slices=100, cut_left=0, 
                    cut_right=rf.t_rf[0, 0]), 
                    FitOptions=FitOptions(fit_option='rms'))

# !~~~~~~~~~~~~~~~~ Trackers ~~~~~~~~~~~~~~~! #
rf_station_tracker = RingAndRFTracker(rf, beam, Profile=profile)
tracker = FullRingAndRF([rf_station_tracker])
bigaussian(ring, rf, beam, tau_0 /4., 0.0014748188 * beam.energy)
# bigaussian(ring, rf, beam, tau_0 /4)

profile.track()
print(profile.bunchLength, tau_0)
print(profile.bunchLength * c / 4., tau_0  * c / 4., np.std(beam.dt) * c)

print('sigmaE: ', np.std(beam.dE) / beam.energy)


1.813297145674646e-11 1.72e-11
0.0013590320209654655 0.0012891075693999999 0.0012888973932887809
sigmaE:  0.0014753375220334577


In [26]:
from lib.formulary import EnergySpread

EnergySpread(C, h, beam.energy * 1e-9, twiss['slip'], tau_0  * c / 4., beam.beta, Vrf * 1e-9, 0, 1)

0.001404502808484857